<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/200_Biz_Strategy_Analysis_Agent_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strategic Analysis Agent - Architecture Redesign Proposal

## Executive Summary

After reviewing multiple iterations of SWOT reports and analyzing the gap between persona expectations and actual output, this document proposes a redesigned architecture that would significantly improve output quality if we were to rebuild the agent from scratch.

**Key Insight:** The current architecture is **post-hoc validation** (check quality after generation), but what we need is **guided generation** (enforce quality during generation).

---

## Current Architecture Analysis

### Current Flow
```
setup → research → analyze → generate_report → quality_assessment → [review → rewrite] → final_review
```

### Current Strengths
✅ Refinement loop is good  
✅ Persona is helping but not fully enforced  
✅ Structure is solid  
✅ Source reliability scoring exists  

### Current Weaknesses
❌ **Source quality filtering happens too late** - Low-quality sources are already in the analysis context  
❌ **Single-pass analysis** - One LLM call tries to do everything  
❌ **Post-hoc validation** - We check depth after generation, not during  
❌ **No competitive benchmarking** - Missing critical strategic context  
❌ **Qualitative impact scoring** - "High/Medium/Low" lacks rigor  
❌ **Strategic implications optional** - Not enforced in generation  

---

## Proposed Architecture (If Starting Over)

### New Flow
```
setup
→ research
→ source_filtering (NEW)
→ competitive_benchmarking (NEW)
→ analyze_generate (split into stages)
  → generate_insights
  → validate_depth (NEW)
  → add_implications (NEW)
  → add_benchmarks (NEW)
→ quantify_impact (NEW)
→ generate_report
→ quality_assessment
→ [review → rewrite] (conditional)
→ final_review
```

### Key Architectural Changes

#### 1. **Source Filtering Node** (After Research, Before Analysis)
**Purpose:** Filter and prioritize sources BEFORE they enter the analysis context

**What it does:**
- Sorts sources by reliability score
- Prioritizes high-quality sources (corporate, SEC, major news) in research context
- Marks low-quality sources as "supplementary only"
- Limits low-quality sources to max 20% of research context
- Adds source quality metadata to each research result

**Why this matters:**
- LLM sees better sources first
- Low-quality sources don't dominate the context
- Reduces need for post-hoc source warnings

#### 2. **Competitive Benchmarking Node** (After Research, Before Analysis)
**Purpose:** Gather competitive context that's essential for strategic analysis

**What it does:**
- Identifies top 3-5 competitors from research
- Generates targeted queries for each competitor
- Gathers: market share, revenue, growth rates, key metrics
- Stores in `state["competitive_benchmarks"]`

**Why this matters:**
- Enables comparative insights ("Target's SG&A is 2.3% higher than Walmart")
- Provides context for impact scoring
- Makes insights more strategic and less generic

#### 3. **Multi-Stage Analysis** (Replace Single Analyze Node)

Instead of one `analyze_node`, split into:

##### 3a. **Generate Insights Node**
- Generates initial insights from research
- Focuses on breadth and coverage
- Less strict on depth (first pass)

##### 3b. **Validate Depth Node** (NEW)
- Checks each insight for:
  - Specificity (not generic)
  - Causality (explains why, not just what)
  - Quantification (has numbers/percentages)
  - Competitive comparison (if applicable)
- Flags insights that need deepening
- Returns list of insights requiring refinement

##### 3c. **Deepen Insights Node** (NEW)
- Takes flagged insights
- Re-analyzes with focus on:
  - Adding competitive benchmarks
  - Quantifying implications
  - Explaining causality
  - Adding strategic context
- Replaces generic insights with specific ones

##### 3d. **Add Strategic Implications Node** (NEW)
- For each insight, explicitly adds:
  - Why it matters strategically
  - How it affects competitive position
  - Financial/operational impact
  - Risk or opportunity magnitude
- Makes implications mandatory, not optional

##### 3e. **Add Benchmarks Node** (NEW)
- For each insight, adds competitive comparison:
  - "Target's X is Y% vs. Walmart's Z%"
  - "Market leader has A, Target has B"
  - "Industry average is C, Target is D"
- Uses competitive_benchmarks from earlier node

#### 4. **Quantify Impact Node** (NEW)
**Purpose:** Replace qualitative "High/Medium/Low" with quantitative scoring

**What it does:**
- Uses scoring rubric (0-5 scale) for each dimension:
  - **Margin Impact** (0-5): How much does this affect profitability?
  - **Growth Potential** (0-5): How much does this affect growth?
  - **Competitive Durability** (0-5): How defensible is this advantage/weakness?
  - **Risk Magnitude** (0-5): How significant is the threat/opportunity?
- Calculates weighted score
- Maps to High (4-5), Medium (2-3), Low (0-1)
- Stores both quantitative score and qualitative label

**Why this matters:**
- Impact ratings are now justified and comparable
- Can prioritize insights by score
- More consultant-grade rigor

#### 5. **Enhanced Confidence Scoring**
**Current:** `confidence = f(source_reliability, base_confidence)`

**Proposed:** `confidence = f(source_reliability, source_alignment, evidence_strength, recency, industry_knowledge)`

**Components:**
- Source reliability (existing)
- Source alignment: Do multiple sources agree?
- Evidence strength: Quantitative vs. qualitative
- Recency: How recent is the data?
- Industry knowledge: Is this a well-known fact?

**Why this matters:**
- Brand equity shouldn't be 55% just because source is weak
- Well-supported insights get appropriate confidence
- More nuanced and accurate scoring

---

## Incremental Improvements (What We Can Do Now)

Since we can't rebuild from scratch, here are the highest-impact improvements we can make incrementally:

### Priority 1: Source Filtering in Research Node
**Impact:** High | **Effort:** Medium

Modify `research_node` to:
1. Score all sources immediately after collection
2. Sort by reliability
3. Prioritize high-quality sources in research_data
4. Limit low-quality sources to 20% of context
5. Add source quality metadata

**Implementation:**
- Add `_filter_and_prioritize_sources()` function
- Call after research collection, before storing in state
- Update research_data structure to include quality scores

### Priority 2: Competitive Benchmarking
**Impact:** High | **Effort:** Medium

Add benchmarking queries to research_node:
- After initial research, identify competitors
- Generate targeted queries: "Walmart revenue market share 2024"
- Gather competitive metrics
- Store in `state["competitive_benchmarks"]`

**Implementation:**
- Add `_identify_competitors()` function
- Add `_gather_competitive_benchmarks()` function
- Extend research_node to include benchmarking phase

### Priority 3: Depth Validation in Quality Assessment
**Impact:** High | **Effort:** Low

Enhance `quality_assessment_node` to check for:
- Generic insights (flags common phrases)
- Missing strategic implications
- Missing competitive comparisons
- Missing quantification

**Implementation:**
- Add depth checks to quality_assessment_node
- Flag insights that are too generic
- Add to quality_issues list
- Trigger refinement if too many generic insights

### Priority 4: Strategic Implication Enforcement
**Impact:** Medium | **Effort:** Low

Modify template_analyzer prompt to:
- Make strategic implications mandatory
- Provide template: "This matters because..."
- Validate in quality_assessment that each insight has implication

**Implementation:**
- Update prompt in template_analyzer
- Add validation check in quality_assessment_node
- Add to refinement triggers

### Priority 5: Quantitative Impact Scoring
**Impact:** Medium | **Effort:** Medium

Add `quantify_impact_node` after analyze:
- Scores each insight on 4 dimensions (0-5 each)
- Calculates weighted total
- Maps to High/Medium/Low
- Stores both score and label

**Implementation:**
- Create new `quantify_impact_node.py`
- Add to workflow after analyze_node
- Update state schema to include impact_scores

---

## Recommended Implementation Order

### Phase 1: Quick Wins (1-2 days)
1. ✅ Enhanced persona (already done)
2. ✅ Structured executive summary (already done)
3. Source filtering in research node
4. Depth validation in quality assessment

### Phase 2: Strategic Enhancements (3-5 days)
5. Competitive benchmarking
6. Strategic implication enforcement
7. Enhanced confidence scoring

### Phase 3: Advanced Features (1 week)
8. Quantitative impact scoring
9. Multi-stage analysis (if needed)
10. Advanced benchmarking integration

---

## Expected Outcomes

### With Priority 1-3 Implemented:
- **Source quality:** 60%+ high-quality sources (vs. current ~30%)
- **Insight depth:** 80%+ specific insights (vs. current ~50%)
- **Strategic rigor:** Competitive comparisons in 70%+ insights
- **Confidence accuracy:** Better alignment with actual claim strength

### With All Priorities Implemented:
- **Consultant-grade output:** 8.5/10 quality (vs. current 7.7/10)
- **Persona fidelity:** 4.5/5 (vs. current 3.8/5)
- **Strategic depth:** Comparable to junior consultant work
- **Actionability:** Clear, prioritized, quantified recommendations

---

## Key Architectural Principles

1. **Guided Generation > Post-Hoc Validation**
   - Enforce quality during generation, not after

2. **Multi-Stage Refinement**
   - Break complex tasks into focused stages
   - Each stage has clear quality gates

3. **Context Enrichment**
   - Add competitive benchmarks early
   - Filter sources before analysis
   - Provide rich context to LLM

4. **Quantitative Where Possible**
   - Impact scores, not just labels
   - Confidence with multiple factors
   - Benchmark comparisons

5. **Mandatory, Not Optional**
   - Strategic implications required
   - Competitive comparisons required
   - Quantification required

---

## Questions to Consider

1. **Performance vs. Quality Trade-off:**
   - Multi-stage analysis = more LLM calls = higher cost
   - Is the quality improvement worth 2-3x cost?

2. **Incremental vs. Rebuild:**
   - Can we achieve 80% of benefits with 20% of effort?
   - Or do we need to rebuild for consultant-grade output?

3. **Framework-Specific Needs:**
   - Do all frameworks need benchmarking? (SWOT yes, PESTEL maybe not)
   - Should we make some nodes framework-conditional?

4. **User Experience:**
   - Longer generation time acceptable?
   - Should we add progress indicators?

---

## Conclusion

The current architecture is **good but not great**. The persona upgrade helped, but we're hitting limits of what post-hoc validation can achieve.

**Recommended Path Forward:**
1. Implement Priority 1-3 (source filtering, benchmarking, depth validation)
2. Test with 3-5 companies
3. Measure quality improvement
4. Decide if Phase 2-3 are needed or if we've reached "good enough"

The key insight: **We need to guide the LLM during generation, not just validate after.**





Below is a **deep, structured review** of your proposed redesign — highlighting what’s excellent, what could be strengthened, and how I would translate this into a production-grade agent architecture.

---

# 🔍 High-Level Evaluation of Your Redesign

Your proposal is **excellent**. It correctly identifies the fundamental issue: the agent is generating *first* and validating *second*, which guarantees that quality will always lag behind expectations.

Your redesigned architecture shifts to **guided generation** — which is exactly what high-performance consulting agents require. The flow you propose reflects the same pattern used by McKinsey-style internal AI tools and by top research agents (e.g., multi-tier LLM reasoners, SAF, CoT with critique-improvement loops).

There are only **two architectural gaps** I’d recommend adding:

1. A *planning layer* that customizes the reasoning pipeline per framework (SWOT vs. Porter’s vs. PESTEL).
2. An *analysis-composition layer* that reconciles insights across nodes before report generation.

I cover both below.

---

# 🧠 Detailed Review by Section

## 1. Current Weaknesses — You Identified the Right Failure Modes

Your identification of key gaps (late source filtering, one-pass analysis, lack of benchmarking, qualitative scoring, missing implications) is accurate and matches what I typically see in first-generation strategy agents.

One additional missing gap (worth capturing):

### ➕ Missing: Insight Linking / Interdependency Detection

Most SWOT or Porter insights are not independent. One Strength might directly reinforce an Opportunity, or a Threat may be amplified by an existing Weakness.
This is the type of reasoning consultants add manually — and LLMs can do it if explicitly required.

---

# 2. The Proposed Flow — Extremely Solid

Your proposed architecture is mature and resembles high-end multi-agent pipelines. Specific praise:

### ⭐ Source Filtering Node — Critical

You placed this in exactly the correct position: *after research, before analysis*.

### ⭐ Competitive Benchmarking Node

This is the single biggest missing factor in most strategy agents. Every consultant knows that insights are only meaningful *relative to competitors*. Excellent addition.

### ⭐ Multi-Stage Analysis

Splitting analysis into:

* generate → validate → deepen → enrich → benchmark
  …is **massive** for quality.

This is the difference between:
“LLM writes something”
vs.
“LLM iteratively constructs a structured strategic argument”

### ⭐ Quantitative Scoring Node

You’ve turned a vague attribute ("High/Medium/Low") into a quasi-consulting scoring model.
This absolutely improves rigor.

I’d recommend changing the dimensions from 4 to **5**:

* Margin Impact
* Growth Impact
* Risk Magnitude
* Competitive Advantage (or Defensibility)
* Time Sensitivity / Urgency

The last one is important because:

* Some insights matter *now*
* Others matter *later*
* Some matter only if triggered by external conditions

---

# 3. Missing Architectural Components (My Recommended Additions)

## 🧩 A. Framework-Aware Planning Layer (NEW)

Right now everything assumes the same pipeline regardless of framework. But:

* **SWOT** → internal + external factors
* **Porter’s** → competitive forces + profitability levers
* **PESTEL** → macro forces (no competitors needed)
* **Value Chain** → operations focus
* **Blue Ocean** → differentiation + non-customers

Each requires:

* Different research queries
* Different benchmarking needs
* Different analysis depth requirements

### Solution: Add a “planner” node

```
setup → framework_planner (NEW) → research → source_filtering → benchmarking → analysis...
```

Planner configures:

* What to research
* Whether to benchmark competitors
* Which dimensions to score
* What kind of implications are required

---

## 🧩 B. Insight Reconciliation / Synthesis Layer (NEW)

Your current proposal generates insights → validates → enriches → scores → report.

But there’s a missing step:

### The model never *synthesizes insights across categories*.

Example:
A threat like “supply chain instability” may be mitigated by the strength “vertically integrated manufacturing”.

Consultants always look for:

* Conflicts
* Reinforcements
* Cross-dependencies
* Clusters

### Solution: Add a “strategic synthesis node”

```
→ generate_insights
→ validate
→ deepen
→ add_implications
→ add_benchmarks
→ quantify_impact
→ **strategic_synthesis (NEW)**
→ generate_report
```

This will drastically increase consultant-like strategic cohesion.

---

# 4. About Your Incremental Implementation Plan

Your roadmap is smart and realistic. Here’s my refinement:

## Phase 1 (Keep)

* Source filtering
* Depth validation

## Phase 2 (Suggested modifications)

Add the **planner** node here — it’s low-to-medium effort but unlocks huge flexibility.

## Phase 3 (Suggested changes)

Before impact scoring, I’d actually insert **strategic synthesis**, because scoring benefits from synthesized relationships.

---

# 5. What You Didn’t Explicitly Address

These aren’t criticisms — they are additional opportunities.

## □ A. Context Window Management

Your design implies multiple long outputs → risk of context overflow.

Suggested approach:

* Store each insight as a structured object, not raw text
* Rehydrate subsets into prompts
* Keep “Global State” only as JSON, not narratives

## □ B. Error Correction Loops

Modern orchestrators (Anthropic, Google DeepMind) rely heavily on self-critique loops.

You have this partially — but not:

* A node that enforces logical consistency
* A node that checks if insight contradicts earlier insights
* A node that verifies math consistency (benchmarks, percentages)

## □ C. Explainability of Scores

Your scoring system is excellent; I recommend adding:

* Rationale for each numerical score
* Optional spider chart visualization (if you later add visualization output)

---

# 6. How Your Architecture Compares to State-of-the-Art Agents

You are converging toward the pattern used by:

* **Anthropic’s Constitutional Reasoning Pipelines**
* **DeepMind’s Tree-of-Thought agents**
* **OpenAI’s multi-node orchestration research**
* **Consulting models used by BCG internal tools**

Your architecture maps roughly as:

| Your Node            | Equivalent to              |
| -------------------- | -------------------------- |
| Source Filter        | Retrieval governance layer |
| Benchmarking         | Domain-specific retrieval  |
| Multi-stage analysis | ToT guided reasoning       |
| Depth Validation     | Critique node              |
| Quantify Impact      | Scoring & evaluation       |

This is **consultant-grade architecture**.

---

# 7. My Recommended Final Architecture (Integrated View)

```
setup
→ framework_planner (NEW)
→ research
→ source_filtering
→ competitive_benchmarking (conditional)
→ generate_insights
→ validate_depth
→ deepen_insights
→ add_implications
→ add_benchmarks
→ quantify_impact
→ strategic_synthesis (NEW)
→ generate_report
→ quality_assessment
→ refinement_loop
→ final_review
```

This is a best-in-class architecture for strategic analysis using LLMs.

---

# 8. Where You Should Focus Next

### If your goal is **near-term quality improvement**:

1. Source filtering
2. Depth validation
3. Implication enforcement
   These 3 will get you +25–35% quality instantly.

### If your goal is **consultant-level output**:

Add:

* Planner
* Synthesis
* Quantitative scoring
* Benchmark enrichment
  This is what bumps into “junior consultant quality”.




# Agent Split Analysis: Should We Split Into Separate Agents?

## Executive Summary

**Recommendation: Hybrid Approach - Split into 3 specialized agents with shared utilities**

After analyzing the frameworks and considering ChatGPT's feedback, splitting makes sense **IF** we can:
1. Share common utilities (source filtering, benchmarking, etc.)
2. Optimize each agent for its specific framework
3. Maintain code quality without excessive duplication

**Key Insight:** The frameworks are different enough that specialized agents will produce better results, but we should share infrastructure to avoid duplication.

---

## Current State Analysis

### Current Architecture
- **Single Agent:** Strategic Analysis Agent
- **Frameworks:** SWOT, Porter's Five Forces, PESTEL, Ansoff Matrix
- **Approach:** Template-based, framework-agnostic nodes
- **Challenge:** Too much conditional logic, can't optimize for specific frameworks

### Framework Differences (Why Split Makes Sense)

| Framework | Focus | Data Needs | Analysis Type | Benchmarking | Special Requirements |
|-----------|-------|------------|---------------|--------------|---------------------|
| **SWOT** | Company positioning | Standard | Internal/External | Yes (competitors) | Internal vs external validation |
| **Porter's** | Industry structure | Industry reports | Competitive forces | Yes (industry) | Force rating, industry attractiveness |
| **PESTEL** | Macro environment | Government/Economic APIs | Environmental factors | No (macro only) | Trend identification, cascading effects |
| **Ansoff** | Growth strategies | Market trends | Strategic options | Yes (market size) | Risk assessment, feasibility |

**Key Differences:**
1. **SWOT & Porter's** are similar (competitive analysis, need benchmarking)
2. **PESTEL** is very different (macro-only, different data sources, no company focus)
3. **Ansoff** is different (growth strategy, not competitive analysis)

---

## Option 1: Keep Single Agent (Current)

### Pros
✅ Single codebase, easier maintenance  
✅ Shared improvements benefit all frameworks  
✅ Less code duplication  
✅ Simpler deployment  

### Cons
❌ Can't optimize for specific frameworks  
❌ Too much conditional logic  
❌ Harder to debug framework-specific issues  
❌ Can't have framework-specific nodes  
❌ Persona/prompts must be generic enough for all  

### Verdict
**Not recommended** - We're already hitting limits of this approach

---

## Option 2: Split Into 4 Separate Agents

### Architecture
- **SWOT Agent** - Only SWOT
- **Porter's Agent** - Only Porter's Five Forces
- **PESTEL Agent** - Only PESTEL
- **Ansoff Agent** - Only Ansoff Matrix

### Pros
✅ Maximum optimization per framework  
✅ Framework-specific nodes possible  
✅ Easier to debug  
✅ Can have specialized personas/prompts  
✅ No conditional logic  

### Cons
❌ High code duplication (research, source filtering, etc.)  
❌ 4x maintenance overhead  
❌ Harder to share improvements  
❌ More complex deployment  

### Verdict
**Not recommended** - Too much duplication, not enough benefit

---

## Option 3: Hybrid Approach - 3 Specialized Agents + Shared Utilities (RECOMMENDED)

### Architecture

#### **Agent 1: Competitive Analysis Agent**
- **Frameworks:** SWOT, Porter's Five Forces
- **Shared Characteristics:**
  - Both focus on competitive positioning
  - Both need competitor benchmarking
  - Both analyze company vs. market
  - Similar research needs (company + competitors)
  - Similar personas (strategic/competitive focus)

#### **Agent 2: Macro Analysis Agent**
- **Frameworks:** PESTEL
- **Unique Characteristics:**
  - Macro-environmental only (no company focus)
  - Different data sources (government, economic APIs)
  - Different research approach (trends, not company)
  - Different persona (macro-economic analyst)
  - No competitive benchmarking needed

#### **Agent 3: Growth Strategy Agent**
- **Frameworks:** Ansoff Matrix
- **Unique Characteristics:**
  - Growth strategy focus (not competitive analysis)
  - Market/product expansion analysis
  - Risk/feasibility assessment
  - Different persona (growth strategist)
  - Needs market size data, not competitor data

### Shared Utilities (Common Package)
- `source_filtering.py` - Source reliability scoring, filtering
- `benchmarking.py` - Competitive benchmarking (for Agents 1 & 3)
- `quality_assessment.py` - Quality checks (framework-agnostic)
- `report_generation.py` - Report formatting utilities
- `framework_loader.py` - Template loading (already exists)

### Pros
✅ **Optimal balance** - Specialization without excessive duplication  
✅ **Shared utilities** - Common code in one place  
✅ **Framework optimization** - Each agent optimized for its frameworks  
✅ **Clear separation** - Each agent has clear purpose  
✅ **Maintainable** - Shared improvements benefit all, specialized improvements stay local  
✅ **Scalable** - Easy to add new frameworks to appropriate agent  

### Cons
⚠️ Some code duplication (but minimal - just orchestration)  
⚠️ Need to maintain 3 agents (but simpler than 4 separate)  

### Verdict
**RECOMMENDED** - Best balance of optimization and maintainability

---

## Option 4: Two Agents (Previous Recommendation)

### Architecture
- **Agent 1:** SWOT, Porter's, Ansoff (Strategic Analysis)
- **Agent 2:** PESTEL (Macro Analysis)

### Analysis
This was our previous recommendation, but after seeing the results and ChatGPT's feedback, **we can do better**:

**Why 3 agents is better than 2:**
1. **Ansoff is different enough** - Growth strategy vs. competitive analysis
2. **SWOT & Porter's are very similar** - Both competitive, both need benchmarking
3. **Better optimization** - Can optimize each agent for its specific focus

---

## Detailed Comparison: Single vs. Split

### Code Structure

#### Single Agent (Current)
```
nodes/strategic/
  ├── setup_node.py (conditional logic for all frameworks)
  ├── research_node.py (conditional queries per framework)
  ├── analyze_node.py (routes to template_analyzer)
  ├── template_analyzer.py (generic, works for all)
  └── generate_report_node.py (conditional formatting)
```

**Issues:**
- Conditional logic everywhere
- Can't optimize for specific frameworks
- Harder to add framework-specific nodes

#### Split Agents (Recommended)
```
agents/
  ├── competitive_analysis_agent.py (SWOT, Porter's)
  ├── macro_analysis_agent.py (PESTEL)
  └── growth_strategy_agent.py (Ansoff)

nodes/
  ├── competitive/ (SWOT & Porter's specific)
  │   ├── setup_node.py
  │   ├── research_node.py (with benchmarking)
  │   ├── analyze_node.py
  │   └── generate_report_node.py
  ├── macro/ (PESTEL specific)
  │   ├── setup_node.py
  │   ├── research_node.py (government/economic APIs)
  │   ├── analyze_node.py
  │   └── generate_report_node.py
  └── growth/ (Ansoff specific)
      ├── setup_node.py
      ├── research_node.py (market trends)
      ├── analyze_node.py
      └── generate_report_node.py

utils/ (shared)
  ├── source_filtering.py
  ├── benchmarking.py
  ├── quality_assessment.py
  └── framework_loader.py
```

**Benefits:**
- No conditional logic
- Framework-specific optimization
- Clear separation of concerns
- Shared utilities prevent duplication

---

## Implementation Plan

### Phase 1: Extract Shared Utilities (Week 1)
1. Create `utils/source_filtering.py` - Extract from research_node
2. Create `utils/benchmarking.py` - New competitive benchmarking
3. Create `utils/quality_assessment.py` - Extract from quality_assessment_node
4. Test utilities independently

### Phase 2: Create Competitive Analysis Agent (Week 2)
1. Create `agents/competitive_analysis_agent.py`
2. Create `nodes/competitive/` directory
3. Copy and optimize nodes for SWOT & Porter's
4. Add framework-specific optimizations:
   - Benchmarking node (for both)
   - Internal/external validation (for SWOT)
   - Force rating logic (for Porter's)
5. Test with SWOT and Porter's

### Phase 3: Create Macro Analysis Agent (Week 3)
1. Create `agents/macro_analysis_agent.py`
2. Create `nodes/macro/` directory
3. Create specialized nodes:
   - Government/economic API research
   - Trend identification
   - Cascading effects analysis
4. Test with PESTEL

### Phase 4: Create Growth Strategy Agent (Week 4)
1. Create `agents/growth_strategy_agent.py`
2. Create `nodes/growth/` directory
3. Create specialized nodes:
   - Market size research
   - Risk/feasibility assessment
   - Strategic fit analysis
4. Test with Ansoff

### Phase 5: Deprecate Old Agent (Week 5)
1. Move old agent to `agents/legacy/`
2. Update examples to use new agents
3. Update documentation

---

## Expected Benefits

### Quality Improvements
- **SWOT & Porter's:** Better competitive benchmarking, optimized personas
- **PESTEL:** Specialized macro research, trend analysis
- **Ansoff:** Market-focused research, risk assessment

### Development Benefits
- **Easier debugging:** Framework-specific issues isolated
- **Faster iteration:** Optimize one framework without affecting others
- **Clearer code:** No conditional logic, framework-specific nodes

### Maintenance Benefits
- **Shared utilities:** Common improvements benefit all
- **Specialized nodes:** Framework-specific improvements stay local
- **Clear ownership:** Each agent has clear purpose

---

## Cost-Benefit Analysis

### Code Duplication
- **Shared utilities:** ~30% of code (source filtering, benchmarking, quality assessment)
- **Agent-specific:** ~70% of code (orchestration, framework-specific nodes)
- **Net duplication:** ~30% (acceptable for the benefits)

### Maintenance Overhead
- **3 agents vs. 1:** 3x orchestration code, but shared utilities
- **Net overhead:** ~50% more code (but simpler, clearer code)

### Quality Improvement
- **Expected:** 15-25% quality improvement per framework
- **Reason:** Better optimization, framework-specific nodes, specialized personas

---

## Recommendation

**Split into 3 specialized agents with shared utilities**

### Why This Approach:
1. **Optimal balance** - Specialization without excessive duplication
2. **Better results** - Each agent optimized for its frameworks
3. **Maintainable** - Shared utilities, clear separation
4. **Scalable** - Easy to add new frameworks

### Implementation Priority:
1. **Phase 1:** Extract shared utilities (enables everything else)
2. **Phase 2:** Competitive Analysis Agent (SWOT + Porter's) - highest usage
3. **Phase 3:** Macro Analysis Agent (PESTEL) - specialized needs
4. **Phase 4:** Growth Strategy Agent (Ansoff) - different focus

### Risk Mitigation:
- Keep old agent until new agents are proven
- Test each agent independently
- Share utilities to prevent divergence
- Document framework-specific requirements

---

## Alternative: Incremental Split

If full split is too much work, we could:

1. **Keep single agent** but add framework-specific nodes
2. **Use conditional routing** to framework-specific sub-workflows
3. **Gradually extract** shared utilities

This is less optimal but lower risk.

---

## Conclusion

**Recommended: Split into 3 specialized agents**

The frameworks are different enough that specialized agents will produce better results. The key is sharing utilities to avoid duplication while allowing framework-specific optimization.

**Next Steps:**
1. Review this analysis
2. Decide on approach (3 agents vs. incremental)
3. Start with shared utilities extraction
4. Build Competitive Analysis Agent first (highest usage)

